In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
import os

# Lecture des fichiers 

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
data_dir = '/content/drive/MyDrive/DATASET/dataset_ai4utg/Generation test sans pretty print/GenerationTest/'

In [7]:
list_folder = os.listdir(data_dir)
num_folder = len(list_folder)
num_folder

1000

# Creation des listes 

In [8]:
from tqdm import tqdm

In [9]:
bdd = []
sbt = []

for i  in tqdm (range(1000), desc="loading..."):
  path_iteration = data_dir+'/Dir_Fonction_'+str(i+1)
  bdd_path = path_iteration+"/BDD_Fonction_"+str(i+1)+".txt"
  stb_path = path_iteration+"/SBT_Generation_Output/Fonction_"+str(i+1)+"_SBT.txt"
  bdd_txt = open(bdd_path,'r').read()
  sbt_txt = open(stb_path,'r').read()
  bdd.append(bdd_txt)
  sbt.append(sbt_txt)

loading...: 100%|██████████| 1000/1000 [12:23<00:00,  1.34it/s]


# Transformation en Dataframe

In [1]:
df = pd.DataFrame(bdd, columns=['bdd'])

NameError: ignored

In [ ]:
df['sbt'] = sbt

In [ ]:
df

,bdd,sbt
0,Feature: Check the Fonction_1\n\n\n # Check t...,( MethodDeclaration ( Modifier public ) Modi...
1,Feature: Check the Fonction_2\n\n\n # Check t...,( MethodDeclaration ( Modifier public ) Modi...
2,Feature: Check the Fonction_3\n\n\n # Check t...,( MethodDeclaration ( Modifier public ) Modi...
3,Feature: Check the Fonction_4\n\n\n # Check t...,( MethodDeclaration ( Modifier public ) Modi...
4,Feature: Check the Fonction_5\n\n\n # Check t...,( MethodDeclaration ( Modifier public ) Modi...
...,...,...
995,Feature: Check the Fonction_996\n\n\n # Check...,( MethodDeclaration ( Modifier public ) Modi...
996,Feature: Check the Fonction_997\n\n\n # Check...,( MethodDeclaration ( Modifier public ) Modi...
997,Feature: Check the Fonction_998\n\n\n # Check...,( MethodDeclaration ( Modifier public ) Modi...
998,Feature: Check the Fonction_999\n\n\n # Check...,( MethodDeclaration ( Modifier public ) Modi...


# Split train / test

In [ ]:
train_bdd = df['bdd'].iloc[:95]
test_bdd = df['bdd'].iloc[-5:]

train_sbt = df['sbt'].iloc[:95]
test_sbt = df['sbt'].iloc[-5:]

# Tokenization

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [ ]:
def tokenize(x):
  tokenizer = Tokenizer(filters='!$%&*,./;<>?@[\\]_^`{|}~\t\n')
  tokenizer.fit_on_texts(x)
  sequences = tokenizer.texts_to_sequences(x)
  return tokenizer, sequences

In [ ]:
tokenizer_sbt, sequences_sbt = tokenize(df['sbt'])
tokenizer_bdd, sequences_bdd = tokenize(df['bdd'])

In [ ]:
tokenizer_sbt.word_index

In [ ]:
tokenizer_bdd.word_index

In [ ]:
word = sbt[0].split(' ')
print(len(word))
word = bdd[0].split(' ')
print(len(word))

219
41


# Padding des sequences 

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
max_len_sequence_sbt = max([len(x) for x in sbt])
print('len max sbt :',max_len_sequence_sbt)
max_len_sequence_bdd = max([len(x) for x in bdd])
print('len max bdd :',max_len_sequence_bdd)

len max sbt : 1572
len max bdd : 193


In [ ]:
def padding(x,length):
  padded = pad_sequences(x,maxlen=length,padding='post',truncating='post')
  return padded

In [ ]:
padded_sbt = padding(sequences_sbt,220)
padded_bdd = padding(sequences_bdd,42)
#padded_sbt = np.expand_dims(padded_sbt,axis=-1)
#padded_bdd = np.expand_dims(padded_bdd,axis=-1)


In [ ]:
padded_sbt.shape

(1000, 220)

In [ ]:
tokenizer_sbt.word_index

In [ ]:
for i in padded_sbt[0]:
  print(i)

#Conversion one hot encoding 

In [ ]:
#sbt_token_index = dict([(char, i) for i, char in enumerate(tokenizer_sbt.word_index)])
#bdd_token_index = dict([(char, i) for i, char in enumerate(tokenizer_bdd.word_index)])

In [ ]:
#input_data = np.zeros((len(padded_sbt), max_len_sequence_sbt, len(tokenizer_sbt.word_index)),dtype='float32')
#output_data = np.zeros((len(padded_bdd), max_len_sequence_sbt, len(tokenizer_bdd.word_index)),dtype='float32')


In [ ]:
#padded_sbt[0]

In [ ]:
'''
for i , (input_text, target_text) in enumerate(zip(padded_sbt,padded_bdd)):
  for t, char in enumerate(padded_sbt[i]):
    print(char)
    input_data[i,t,char] = 1.
  break

   ''' 

'\nfor i , (input_text, target_text) in enumerate(zip(padded_sbt,padded_bdd)):\n  for t, char in enumerate(padded_sbt[i]):\n    print(char)\n    input_data[i,t,char] = 1.\n  break\n\n   '

In [ ]:
'''
print(input_data.shape)
print(output_data.shape)
'''

'\nprint(input_data.shape)\nprint(output_data.shape)\n'

# decodage

In [ ]:
def sequence_to_sentence(sequence, tokenizer):
  decoded_sequence = []
  for i in sequence:
    for word, index in tokenizer.word_index.items():
      if i == index:
        decoded_sequence.append(word)
        decoded_sequence.append(' ')
  return ''.join(decoded_sequence)


In [ ]:
padded_bdd[0].shape

(42,)

In [ ]:
print(sequence_to_sentence(padded_sbt[1],tokenizer_sbt))

( methoddeclaration ( modifier public ) modifier public ( modifier static ) modifier static ( simplename fonction 2 ) simplename fonction 2 ( parameter ( primitivetype int ) primitivetype int ( simplename k ) simplename k ) parameter ( parameter ( primitivetype int ) primitivetype int ( simplename m ) simplename m ) parameter ( primitivetype boolean ) primitivetype boolean ( blockstmt ( expressionstmt ( variabledeclarationexpr ( variabledeclarator ( primitivetype int ) primitivetype int ( simplename l ) simplename l ( unaryexpr ( integerliteralexpr 239 ) integerliteralexpr 239 ) unaryexpr ) variabledeclarator ) variabledeclarationexpr ) expressionstmt ( expressionstmt ( variabledeclarationexpr ( variabledeclarator ( primitivetype int ) primitivetype int ( simplename z ) simplename z ( integerliteralexpr 276 ) integerliteralexpr 276 ) variabledeclarator ) variabledeclarationexpr ) expressionstmt ( expressionstmt ( variabledeclarationexpr ( variabledeclarator ( primitivetype boolean ) pr

In [ ]:
print(sequence_to_sentence(padded_bdd[1],tokenizer_bdd))

feature: check the fonction 2 # check the result to true scenario: bdd fonction 2 1 given k = -363 and m = 399 when i call fonction 2 then the result should be true 


# Creation du Modele

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GRU, Dropout, TimeDistributed, LSTM, Bidirectional, Embedding, RepeatVector
from tensorflow.keras.losses import sparse_categorical_crossentropy

In [ ]:
model = Sequential()
model.add(Embedding(len(tokenizer_sbt.word_index)+1,256))
model.add(Bidirectional(LSTM(256, return_sequences=True)))
model.add(Bidirectional(LSTM(256, return_sequences=True)))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(tokenizer_bdd.word_index)+1, activation='softmax'))
model.compile(loss=sparse_categorical_crossentropy, optimizer='adam',metrics=['accuracy'])

In [ ]:
model.fit(padded_sbt,padded_bdd,batch_size = 32, epochs=30)

Epoch 1/30


ValueError: ignored

In [ ]:
model.save('v1.h5')

In [ ]:
prediction = model.predict(padded_sbt[5])


In [ ]:
prediction.shape

(2057, 1, 4717)

In [ ]:
prediction = np.squeeze(prediction, axis=1)


In [ ]:
prediction[0]

array([0.00046511, 0.00043722, 0.00048602, ..., 0.00019981, 0.00019607,
       0.00022998], dtype=float32)

In [ ]:
test = np.argmax(prediction[0])

In [ ]:
test

2

In [ ]:
prediction.shape

(2057, 4717)

In [ ]:
  pred_sequence = []
  for i in range(len(prediction)):
      pred_sequence.append(np.argmax(prediction[i]))

In [ ]:
pred_sequence

[2,
 5,
 2,
 5,
 5,
 0,
 5,
 5,
 46,
 2,
 5,
 46,
 0,
 5,
 46,
 46,
 2,
 2,
 2,
 0,
 2,
 2,
 2,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 2,
 2,
 0,
 0,
 2,
 0,
 0,
 0,
 2,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 2,
 2,
 0,
 0,
 2,
 2,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 2,
 2,
 2,
 2,
 2,
 2,
 0,
 2,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 2,
 2,
 0,
 0,
 2,
 2,
 2,
 0,
 2,
 2,
 2,
 0,
 2,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 2,
 2,
 2,
 0,
 2,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 2,
 2,
 0,
 0,
 2,
 0,
 2,
 0,
 2,
 2,
 2,
 0,
 2,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 2,
 2,
 2,
 0,
 2,
 0,
 2,
 0,
 0,
 0,
 0,
 2,
 2,
 2,
 0,
 0,
 2,
 0,
 2,
 2,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 2,
 0,
 2,
 0,
 2,
 0,
 2,
 2,
 0,
 0,
 2,
 0,
 0,
 0,
 2,
 0,
 2,
 2,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 2,
 2,
 2,
 0,
 2,
 2,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 2,
 0,
 2,
 0,
 2,
 2,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 5,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


# Test

In [ ]:
for i in range(10):
  true_translation = df['bdd'][i]
  #original_text = lines_en[i]
  #print(original_text)
  print(' true bdd : \n',true_translation)

  #new_tmp = np.expand_dims(tmp_x[i],axis=0)
  #print(new_tmp.shape)
  prediction = model.predict(padded_sbt[i])
  prediction = np.squeeze(prediction, axis=1)
  pred_sequence = []
  for i in range(len(prediction)):
      pred_sequence.append(np.argmax(prediction[i]))
  true_pred = sequence_to_sentence(pred_sequence, tokenizer_bdd)
  print('prediction bdd: \n',true_pred)
  print("\n")



 true bdd : 
 Feature: Check the Fonction_1


  # Check the result to True
  Scenario: bdd_Fonction_1_1
    Given s = 166 and l = -1961
    When i call Fonction_1
    Then the result should be True

prediction bdd: 
 check = check = = = = -991 check = -991 = -991 -991 check check feature: check check check check check check check check check check check check check check check check check check check check check check check check check check check check check check check check check check check check check check check check check check check check check check check check check check check check check check check check check check check check check check check check check check check check check check check check check check check check check check check check check check check check check check = 


 true bdd : 
 Feature: Check the Fonction_2


  # Check the result to True
  Scenario: bdd_Fonction_2_1
    Given t = 960 and e = -953
    When i call Fonction_2
    Then the result should 